# Notebook 19: Branch-Level Forecast Validation

This notebook validates ML forecasts vs Traditional forecasts at the **Betriebszentrale (dispatch center) level**.

**Approach**: Proportional Distribution Method
- Takes company-level forecasts from Notebook 18
- Distributes them to branches based on historical proportions (2022-2024)
- Compares branch-level accuracy: ML vs Traditional method

**Key Question**: Do some branches benefit more from ML forecasting than others?

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print('Notebook 19: Branch-Level Forecast Validation')
print('='*80)

Notebook 19: Branch-Level Forecast Validation


## Section 1: Load Data

Load three datasets:
1. **2025 Actual by Branch**: From Notebook 18 (generated with betriebszentrale mapping)
2. **Historical Branch Data** (2022-2024): To calculate branch proportions
3. **Company-Level Forecasts**: ML and Traditional forecasts to distribute

In [2]:
print('Loading data...')
print('='*80)

# 1. Load 2025 actual data by branch (from Notebook 18)
print('\n1. Loading 2025 actual data by branch...')
actual_2025_bz = pd.read_csv('../data/processed/2025_actual_by_branch.csv')
actual_2025_bz['date'] = pd.to_datetime(actual_2025_bz['date'])
print(f'   Shape: {actual_2025_bz.shape}')
print(f'   Branches: {actual_2025_bz["betriebszentrale"].nunique()}')
print(f'   Months: {actual_2025_bz["date"].nunique()}')
print(f'   Date range: {actual_2025_bz["date"].min()} to {actual_2025_bz["date"].max()}')

# 2. Load historical branch data (2022-2024)
print('\n2. Loading historical branch data (2022-2024)...')
historical_bz = pd.read_csv('../data/processed/monthly_aggregated_full_bz.csv')
historical_bz['date'] = pd.to_datetime(historical_bz['date'])
print(f'   Shape: {historical_bz.shape}')
print(f'   Branches: {historical_bz["betriebszentrale"].nunique()}')
print(f'   Date range: {historical_bz["date"].min()} to {historical_bz["date"].max()}')

# 3. Load company-level ML forecasts
print('\n3. Loading company-level ML forecasts...')
forecast_ml = pd.read_csv('../data/processed/consolidated_forecast_2025.csv')
forecast_ml['date'] = pd.to_datetime(forecast_ml['date'])
# Filter to Jan-Sep 2025
forecast_ml = forecast_ml[(forecast_ml['date'] >= '2025-01-01') & (forecast_ml['date'] <= '2025-09-01')]
print(f'   Shape: {forecast_ml.shape}')
print(f'   Months: {len(forecast_ml)}')

# 4. Calculate traditional (2024÷12) baseline
print('\n4. Calculating traditional baseline (2024 total ÷ 12)...')
data_2024 = historical_bz[historical_bz['date'].dt.year == 2024]
total_orders_2024 = data_2024['total_orders'].sum()
revenue_total_2024 = data_2024['revenue_total'].sum()

human_orders_monthly = total_orders_2024 / 12
human_revenue_monthly = revenue_total_2024 / 12

print(f'   2024 Total Orders: {total_orders_2024:,.0f}')
print(f'   2024 Total Revenue: CHF {revenue_total_2024:,.2f}')
print(f'   Human Forecast (monthly): {human_orders_monthly:,.0f} orders, CHF {human_revenue_monthly:,.2f}')

print('\n' + '='*80)
print('✓ All data loaded successfully')

Loading data...

1. Loading 2025 actual data by branch...


   Shape: (108, 8)
   Branches: 12
   Months: 9
   Date range: 2025-01-01 00:00:00 to 2025-09-01 00:00:00

2. Loading historical branch data (2022-2024)...


   Shape: (369, 15)
   Branches: 11
   Date range: 2022-01-01 00:00:00 to 2024-12-01 00:00:00

3. Loading company-level ML forecasts...
   Shape: (9, 11)
   Months: 9

4. Calculating traditional baseline (2024 total ÷ 12)...
   2024 Total Orders: 1,641,250
   2024 Total Revenue: CHF 157,996,583.14
   Human Forecast (monthly): 136,771 orders, CHF 13,166,381.93

✓ All data loaded successfully


## Section 2: Calculate Historical Branch Proportions

Calculate each branch's average share of company total (2022-2024)

In [3]:
print('Calculating historical branch proportions (2022-2024)...')
print('='*80)

# Calculate company-level monthly totals
historical_company = historical_bz.groupby('date').agg({
    'total_orders': 'sum',
    'revenue_total': 'sum'
}).reset_index()

# Merge branch data with company totals
historical_with_totals = historical_bz.merge(
    historical_company[['date', 'total_orders', 'revenue_total']],
    on='date',
    suffixes=('_branch', '_company'),
    how='left'
)

# Calculate proportions for each branch-month
historical_with_totals['orders_pct'] = (
    historical_with_totals['total_orders_branch'] / historical_with_totals['total_orders_company'] * 100
)
historical_with_totals['revenue_pct'] = (
    historical_with_totals['revenue_total_branch'] / historical_with_totals['revenue_total_company'] * 100
)

# Calculate average proportion per branch
branch_proportions = historical_with_totals.groupby('betriebszentrale').agg({
    'orders_pct': 'mean',
    'revenue_pct': 'mean',
    'total_orders_branch': 'mean',  # Average monthly orders
    'revenue_total_branch': 'mean'  # Average monthly revenue
}).reset_index()

# Rename columns
branch_proportions.columns = ['betriebszentrale', 'orders_pct', 'revenue_pct', 'avg_monthly_orders', 'avg_monthly_revenue']

# Sort by revenue proportion (descending)
branch_proportions = branch_proportions.sort_values('revenue_pct', ascending=False)

print(f'\nBranch proportions calculated for {len(branch_proportions)} branches')
print(f'\nTop branches by revenue share (2022-2024 average):')
display(branch_proportions.head(10))

print(f'\nValidation: Total proportions should sum to ~100%')
print(f'  Orders: {branch_proportions["orders_pct"].sum():.2f}%')
print(f'  Revenue: {branch_proportions["revenue_pct"].sum():.2f}%')

print('\n' + '='*80)

Calculating historical branch proportions (2022-2024)...

Branch proportions calculated for 11 branches

Top branches by revenue share (2022-2024 average):


,betriebszentrale,orders_pct,revenue_pct,avg_monthly_orders,avg_monthly_revenue
9,BZ Sursee,22.068853,37.609613,30280.138889,4.864954e+06
3,BZ Herzogenbuchsee,5.138051,24.466105,7042.833333,3.153532e+06
10,BZ Winterthur,22.275290,9.881167,30543.694444,1.276224e+06
6,BZ Oberbipp,29.399323,9.731556,40295.611111,1.255196e+06
2,B&T Winterthur,3.855796,6.081003,5300.333333,7.854533e+05
5,BZ Landquart,13.268924,5.773921,18185.111111,7.433762e+05
1,B&T Puidoux,1.901372,3.902831,2611.777778,5.041185e+05
8,BZ Sierre,3.927936,1.354193,5436.111111,1.775061e+05
4,BZ Intermodal / Rail,0.262756,1.031095,361.083333,1.336161e+05
0,B&T Landquart,0.344657,0.651415,474.555556,8.446936e+04



Validation: Total proportions should sum to ~100%
  Orders: 102.95%
  Revenue: 101.02%



## Section 3: Distribute Forecasts to Branches

Distribute both ML and Traditional forecasts to branches proportionally

In [4]:
print('Distributing forecasts to branches...')
print('='*80)

# Distribute ML forecasts
print('\n1. Distributing ML forecasts...')
ml_by_branch = []
for _, month_forecast in forecast_ml.iterrows():
    for _, bz in branch_proportions.iterrows():
        ml_by_branch.append({
            'date': month_forecast['date'],
            'betriebszentrale': bz['betriebszentrale'],
            'orders_ml': month_forecast['total_orders'] * (bz['orders_pct'] / 100),
            'revenue_ml': month_forecast['revenue_total'] * (bz['revenue_pct'] / 100)
        })

ml_by_branch_df = pd.DataFrame(ml_by_branch)
print(f'   ML forecasts distributed: {ml_by_branch_df.shape[0]} records')
print(f'   (9 months × {len(branch_proportions)} branches)')

# Distribute Traditional forecasts
print('\n2. Distributing Traditional forecasts...')
traditional_by_branch = []
dates_2025 = pd.date_range('2025-01-01', '2025-09-01', freq='MS')
for date in dates_2025:
    for _, bz in branch_proportions.iterrows():
        traditional_by_branch.append({
            'date': date,
            'betriebszentrale': bz['betriebszentrale'],
            'orders_traditional': human_orders_monthly * (bz['orders_pct'] / 100),
            'revenue_traditional': human_revenue_monthly * (bz['revenue_pct'] / 100)
        })

traditional_by_branch_df = pd.DataFrame(traditional_by_branch)
print(f'   Traditional forecasts distributed: {traditional_by_branch_df.shape[0]} records')

# Merge ML and Traditional forecasts
forecasts_by_branch = ml_by_branch_df.merge(
    traditional_by_branch_df,
    on=['date', 'betriebszentrale'],
    how='outer'
)

print(f'\n✓ Combined forecast dataframe: {forecasts_by_branch.shape}')
print('\nSample (first branch, first 3 months):')
display(forecasts_by_branch.head(3))

print('\n' + '='*80)

Distributing forecasts to branches...

1. Distributing ML forecasts...
   ML forecasts distributed: 99 records
   (9 months × 11 branches)

2. Distributing Traditional forecasts...
   Traditional forecasts distributed: 99 records

✓ Combined forecast dataframe: (99, 6)

Sample (first branch, first 3 months):


,date,betriebszentrale,orders_ml,revenue_ml,orders_traditional,revenue_traditional
0,2025-01-01,B&T Landquart,454.808563,78954.942215,471.390598,85767.848902
1,2025-01-01,B&T Puidoux,2509.044506,473043.460821,2600.522694,513861.690399
2,2025-01-01,B&T Winterthur,5088.096012,737049.233955,5273.604801,800648.136242


## Section 4: Calculate Accuracy Metrics by Branch

Calculate MAPE and MAE for each branch

In [5]:
print('Calculating accuracy metrics by branch...')
print('='*80)

def calculate_mape(actual, predicted):
    """Calculate Mean Absolute Percentage Error"""
    return np.mean(np.abs((actual - predicted) / actual)) * 100

def calculate_mae(actual, predicted):
    """Calculate Mean Absolute Error"""
    return np.mean(np.abs(actual - predicted))

# Merge actuals with forecasts
comparison = actual_2025_bz.merge(
    forecasts_by_branch,
    on=['date', 'betriebszentrale'],
    how='inner'
)

print(f'\nMerged comparison data: {comparison.shape}')
print(f'  Branches with data: {comparison["betriebszentrale"].nunique()}')

# Calculate metrics for each branch
branch_results = []
for bz in comparison['betriebszentrale'].unique():
    bz_data = comparison[comparison['betriebszentrale'] == bz]
    
    # Check if we have enough data
    if len(bz_data) < 3:
        print(f'⚠️  Skipping {bz}: only {len(bz_data)} months of data')
        continue
    
    # Orders metrics
    orders_mape_ml = calculate_mape(bz_data['total_orders'], bz_data['orders_ml'])
    orders_mape_traditional = calculate_mape(bz_data['total_orders'], bz_data['orders_traditional'])
    orders_mae_ml = calculate_mae(bz_data['total_orders'], bz_data['orders_ml'])
    orders_mae_traditional = calculate_mae(bz_data['total_orders'], bz_data['orders_traditional'])
    
    # Revenue metrics
    revenue_mape_ml = calculate_mape(bz_data['revenue_total'], bz_data['revenue_ml'])
    revenue_mape_traditional = calculate_mape(bz_data['revenue_total'], bz_data['revenue_traditional'])
    revenue_mae_ml = calculate_mae(bz_data['revenue_total'], bz_data['revenue_ml'])
    revenue_mae_traditional = calculate_mae(bz_data['revenue_total'], bz_data['revenue_traditional'])
    
    # Determine winners
    orders_winner = 'ML' if orders_mape_ml < orders_mape_traditional else 'Traditional'
    revenue_winner = 'ML' if revenue_mape_ml < revenue_mape_traditional else 'Traditional'
    
    # Average monthly values
    avg_monthly_orders = bz_data['total_orders'].mean()
    avg_monthly_revenue = bz_data['revenue_total'].mean()
    
    branch_results.append({
        'betriebszentrale': bz,
        'avg_monthly_orders': avg_monthly_orders,
        'avg_monthly_revenue': avg_monthly_revenue,
        'orders_mape_ml': orders_mape_ml,
        'orders_mape_traditional': orders_mape_traditional,
        'orders_mae_ml': orders_mae_ml,
        'orders_mae_traditional': orders_mae_traditional,
        'orders_winner': orders_winner,
        'orders_improvement_%': ((orders_mape_traditional - orders_mape_ml) / orders_mape_traditional * 100),
        'revenue_mape_ml': revenue_mape_ml,
        'revenue_mape_traditional': revenue_mape_traditional,
        'revenue_mae_ml': revenue_mae_ml,
        'revenue_mae_traditional': revenue_mae_traditional,
        'revenue_winner': revenue_winner,
        'revenue_improvement_%': ((revenue_mape_traditional - revenue_mape_ml) / revenue_mape_traditional * 100),
        'months_of_data': len(bz_data)
    })

branch_results_df = pd.DataFrame(branch_results)

# Sort by revenue (largest branches first)
branch_results_df = branch_results_df.sort_values('avg_monthly_revenue', ascending=False)

print(f'\n✓ Calculated metrics for {len(branch_results_df)} branches')
print('\nBranch-Level Results (sorted by revenue):')
display(branch_results_df[[
    'betriebszentrale', 'avg_monthly_revenue', 
    'orders_mape_ml', 'orders_mape_traditional', 'orders_winner',
    'revenue_mape_ml', 'revenue_mape_traditional', 'revenue_winner'
]])

print('\n' + '='*80)

Calculating accuracy metrics by branch...

Merged comparison data: (99, 12)
  Branches with data: 11

✓ Calculated metrics for 11 branches

Branch-Level Results (sorted by revenue):


,betriebszentrale,avg_monthly_revenue,orders_mape_ml,orders_mape_traditional,orders_winner,revenue_mape_ml,revenue_mape_traditional,revenue_winner
9,BZ Sursee,5.100098e+06,5.807311,7.698275,ML,6.708681,7.977629,ML
3,BZ Herzogenbuchsee,3.189417e+06,8.177340,6.883434,Traditional,9.587396,7.877067,Traditional
6,BZ Oberbipp,1.299130e+06,6.231980,5.964762,Traditional,5.745981,2.271645,Traditional
10,BZ Winterthur,1.246136e+06,5.657193,5.535767,Traditional,5.891182,7.957557,ML
2,B&T Winterthur,8.458191e+05,39.058112,39.102872,ML,8.232898,7.431599,Traditional
5,BZ Landquart,7.107365e+05,5.468875,5.691818,ML,8.425162,7.698809,Traditional
1,B&T Puidoux,4.908903e+05,8.882117,10.961228,ML,6.293407,8.565118,ML
8,BZ Sierre,2.155459e+05,12.426223,12.536178,ML,17.759006,16.817454,Traditional
7,BZ Puidoux,1.889171e+05,62.805899,62.798248,Traditional,62.965170,62.343040,Traditional
4,BZ Intermodal / Rail,1.597710e+05,10.415459,11.308792,ML,14.746138,15.098508,ML


## Section 5: Summary Statistics

In [6]:
print('Summary Statistics')
print('='*80)

print('\n📊 ORDERS FORECAST ACCURACY BY BRANCH')
print('-'*80)
print(f'Average MAPE across all branches:')
print(f'  ML: {branch_results_df["orders_mape_ml"].mean():.2f}%')
print(f'  Traditional: {branch_results_df["orders_mape_traditional"].mean():.2f}%')
print(f'\nBranches where ML wins: {(branch_results_df["orders_winner"] == "ML").sum()} / {len(branch_results_df)}')
print(f'Branches where Traditional wins: {(branch_results_df["orders_winner"] == "Traditional").sum()} / {len(branch_results_df)}')

print('\n📊 REVENUE FORECAST ACCURACY BY BRANCH')
print('-'*80)
print(f'Average MAPE across all branches:')
print(f'  ML: {branch_results_df["revenue_mape_ml"].mean():.2f}%')
print(f'  Traditional: {branch_results_df["revenue_mape_traditional"].mean():.2f}%')
print(f'\nBranches where ML wins: {(branch_results_df["revenue_winner"] == "ML").sum()} / {len(branch_results_df)}')
print(f'Branches where Traditional wins: {(branch_results_df["revenue_winner"] == "Traditional").sum()} / {len(branch_results_df)}')

# Check for branches with large differences
print('\n🔍 BRANCHES WITH LARGEST ML ADVANTAGE (Orders):')
top_ml_branches = branch_results_df.nlargest(5, 'orders_improvement_%')[[
    'betriebszentrale', 'orders_mape_ml', 'orders_mape_traditional', 'orders_improvement_%'
]]
display(top_ml_branches)

print('\n🔍 BRANCHES WHERE TRADITIONAL PERFORMS BEST (Orders):')
top_traditional_branches = branch_results_df.nsmallest(5, 'orders_improvement_%')[[
    'betriebszentrale', 'orders_mape_ml', 'orders_mape_traditional', 'orders_improvement_%'
]]
display(top_traditional_branches)

print('\n' + '='*80)

Summary Statistics

📊 ORDERS FORECAST ACCURACY BY BRANCH
--------------------------------------------------------------------------------
Average MAPE across all branches:
  ML: 16.45%
  Traditional: 16.88%

Branches where ML wins: 7 / 11
Branches where Traditional wins: 4 / 11

📊 REVENUE FORECAST ACCURACY BY BRANCH
--------------------------------------------------------------------------------
Average MAPE across all branches:
  ML: 14.37%
  Traditional: 14.26%

Branches where ML wins: 5 / 11
Branches where Traditional wins: 6 / 11

🔍 BRANCHES WITH LARGEST ML ADVANTAGE (Orders):


,betriebszentrale,orders_mape_ml,orders_mape_traditional,orders_improvement_%
9,BZ Sursee,5.807311,7.698275,24.563480
1,B&T Puidoux,8.882117,10.961228,18.967870
4,BZ Intermodal / Rail,10.415459,11.308792,7.899459
0,B&T Landquart,15.972381,17.237291,7.338216
5,BZ Landquart,5.468875,5.691818,3.916903



🔍 BRANCHES WHERE TRADITIONAL PERFORMS BEST (Orders):


,betriebszentrale,orders_mape_ml,orders_mape_traditional,orders_improvement_%
3,BZ Herzogenbuchsee,8.177340,6.883434,-18.797389
6,BZ Oberbipp,6.231980,5.964762,-4.479950
10,BZ Winterthur,5.657193,5.535767,-2.193474
7,BZ Puidoux,62.805899,62.798248,-0.012184
2,B&T Winterthur,39.058112,39.102872,0.114468


## Section 6: Visualizations

Create interactive charts showing branch-level accuracy

In [7]:
print('Creating visualizations...')
print('='*80)

# Visualization 1: Bar chart comparison - Orders MAPE by Branch
fig1 = go.Figure()

# Sort by average revenue (show largest branches first)
chart_data = branch_results_df.sort_values('avg_monthly_revenue', ascending=True)

fig1.add_trace(go.Bar(
    y=chart_data['betriebszentrale'],
    x=chart_data['orders_mape_traditional'],
    name='Traditional (2024÷12)',
    orientation='h',
    marker_color='#FF6B6B',
    text=chart_data['orders_mape_traditional'].apply(lambda x: f'{x:.1f}%'),
    textposition='outside'
))

fig1.add_trace(go.Bar(
    y=chart_data['betriebszentrale'],
    x=chart_data['orders_mape_ml'],
    name='Machine Learning',
    orientation='h',
    marker_color='#4ECDC4',
    text=chart_data['orders_mape_ml'].apply(lambda x: f'{x:.1f}%'),
    textposition='outside'
))

fig1.update_layout(
    title='Total Orders: Forecast Accuracy by Betriebszentrale (MAPE %)',
    xaxis_title='MAPE (%) - Lower is Better',
    yaxis_title='Betriebszentrale',
    barmode='group',
    height=600,
    template='plotly_white',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig1.show()
fig1.write_html('../results/branch_level_orders_mape_comparison.html')
print('✓ Saved: branch_level_orders_mape_comparison.html')

Creating visualizations...


✓ Saved: branch_level_orders_mape_comparison.html


In [8]:
# Visualization 2: Bar chart comparison - Revenue MAPE by Branch
fig2 = go.Figure()

fig2.add_trace(go.Bar(
    y=chart_data['betriebszentrale'],
    x=chart_data['revenue_mape_traditional'],
    name='Traditional (2024÷12)',
    orientation='h',
    marker_color='#FF6B6B',
    text=chart_data['revenue_mape_traditional'].apply(lambda x: f'{x:.1f}%'),
    textposition='outside'
))

fig2.add_trace(go.Bar(
    y=chart_data['betriebszentrale'],
    x=chart_data['revenue_mape_ml'],
    name='Machine Learning',
    orientation='h',
    marker_color='#4ECDC4',
    text=chart_data['revenue_mape_ml'].apply(lambda x: f'{x:.1f}%'),
    textposition='outside'
))

fig2.update_layout(
    title='Revenue Total: Forecast Accuracy by Betriebszentrale (MAPE %)',
    xaxis_title='MAPE (%) - Lower is Better',
    yaxis_title='Betriebszentrale',
    barmode='group',
    height=600,
    template='plotly_white',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig2.show()
fig2.write_html('../results/branch_level_revenue_mape_comparison.html')
print('✓ Saved: branch_level_revenue_mape_comparison.html')

✓ Saved: branch_level_revenue_mape_comparison.html


In [9]:
# Visualization 3: Heatmap showing improvement percentage
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Orders: ML Improvement vs Traditional', 'Revenue: ML Improvement vs Traditional'],
    horizontal_spacing=0.15
)

# Sort by branch size (revenue)
heatmap_data = branch_results_df.sort_values('avg_monthly_revenue', ascending=False)

# Orders heatmap
fig3.add_trace(go.Bar(
    x=heatmap_data['orders_improvement_%'],
    y=heatmap_data['betriebszentrale'],
    orientation='h',
    marker=dict(
        color=heatmap_data['orders_improvement_%'],
        colorscale='RdYlGn',
        cmin=-20,
        cmax=20,
        colorbar=dict(title="% Improvement", x=0.45)
    ),
    text=heatmap_data['orders_improvement_%'].apply(lambda x: f'{x:+.1f}%'),
    textposition='outside',
    showlegend=False
), row=1, col=1)

# Revenue heatmap
fig3.add_trace(go.Bar(
    x=heatmap_data['revenue_improvement_%'],
    y=heatmap_data['betriebszentrale'],
    orientation='h',
    marker=dict(
        color=heatmap_data['revenue_improvement_%'],
        colorscale='RdYlGn',
        cmin=-20,
        cmax=20,
        colorbar=dict(title="% Improvement", x=1.02)
    ),
    text=heatmap_data['revenue_improvement_%'].apply(lambda x: f'{x:+.1f}%'),
    textposition='outside',
    showlegend=False
), row=1, col=2)

# Add zero line
fig3.add_vline(x=0, line_dash='dash', line_color='gray', row=1, col=1)
fig3.add_vline(x=0, line_dash='dash', line_color='gray', row=1, col=2)

fig3.update_layout(
    title_text='<b>ML vs Traditional: Improvement % by Branch</b><br>(Positive = ML Better, Negative = Traditional Better)',
    height=600,
    template='plotly_white'
)

fig3.update_xaxes(title_text='ML Improvement (%)', row=1, col=1)
fig3.update_xaxes(title_text='ML Improvement (%)', row=1, col=2)

fig3.show()
fig3.write_html('../results/branch_level_improvement_heatmap.html')
print('✓ Saved: branch_level_improvement_heatmap.html')

print('\n' + '='*80)
print('✓ All visualizations created successfully')

✓ Saved: branch_level_improvement_heatmap.html

✓ All visualizations created successfully


## Section 7: Save Summary Report

In [10]:
# Save branch-level results to CSV
branch_results_df.to_csv('../results/branch_level_validation_summary.csv', index=False)
print('✓ Saved: branch_level_validation_summary.csv')

# Create executive summary
summary = {
    'Metric': [],
    'ML Avg MAPE (%)': [],
    'Traditional Avg MAPE (%)': [],
    'Branches where ML Wins': [],
    'Branches where Traditional Wins': []
}

# Orders summary
summary['Metric'].append('Total Orders')
summary['ML Avg MAPE (%)'].append(branch_results_df['orders_mape_ml'].mean())
summary['Traditional Avg MAPE (%)'].append(branch_results_df['orders_mape_traditional'].mean())
summary['Branches where ML Wins'].append((branch_results_df['orders_winner'] == 'ML').sum())
summary['Branches where Traditional Wins'].append((branch_results_df['orders_winner'] == 'Traditional').sum())

# Revenue summary
summary['Metric'].append('Revenue Total')
summary['ML Avg MAPE (%)'].append(branch_results_df['revenue_mape_ml'].mean())
summary['Traditional Avg MAPE (%)'].append(branch_results_df['revenue_mape_traditional'].mean())
summary['Branches where ML Wins'].append((branch_results_df['revenue_winner'] == 'ML').sum())
summary['Branches where Traditional Wins'].append((branch_results_df['revenue_winner'] == 'Traditional').sum())

summary_df = pd.DataFrame(summary)
summary_df.to_csv('../results/branch_level_executive_summary.csv', index=False)
print('✓ Saved: branch_level_executive_summary.csv')

print('\n' + '='*80)
print('✓✓✓ BRANCH-LEVEL VALIDATION COMPLETE ✓✓✓')
print('='*80)
print('\nGenerated files in results/:')
print('  1. branch_level_orders_mape_comparison.html')
print('  2. branch_level_revenue_mape_comparison.html')
print('  3. branch_level_improvement_heatmap.html')
print('  4. branch_level_validation_summary.csv')
print('  5. branch_level_executive_summary.csv')
print('='*80)

✓ Saved: branch_level_validation_summary.csv

✓ Saved: branch_level_executive_summary.csv

✓✓✓ BRANCH-LEVEL VALIDATION COMPLETE ✓✓✓

Generated files in results/:
  1. branch_level_orders_mape_comparison.html
  2. branch_level_revenue_mape_comparison.html
  3. branch_level_improvement_heatmap.html
  4. branch_level_validation_summary.csv
  5. branch_level_executive_summary.csv


## Key Insights

This analysis shows whether ML forecasting accuracy varies by branch (Betriebszentrale).

**Questions answered**:
1. Do some branches benefit more from ML forecasting?
2. Are certain branches harder to forecast (higher MAPE for both methods)?
3. Should we use branch-specific forecasting strategies?

**Next steps**:
- Investigate branches where one method significantly outperforms
- Consider branch-specific model training for branches where both methods perform poorly
- Use these insights to refine the hybrid forecasting approach recommended in Notebook 18